In [43]:
import xml.etree.ElementTree as ET

#tree = ET.parse(r'C:\Users\Chris\Documents\projects\train-game\drawing.svg')
tree = ET.parse(r'drawing-nobg.svg')
root = tree.getroot()

root

ns = ns = {'svg': 'http://www.w3.org/2000/svg',
      'ttr': 'https://train-game.github.io',
          'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}


In [44]:
# Copy inkscape labels to element_ids
for node in root.iter():
    label = node.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label')
    if label:
        node.set('id', label)
    

In [45]:
tree.write('processed.svg')

In [35]:
# dir(node)
# node.attrib
# node.set('id', 'banana')
node.get('id')

'banana'

In [19]:
for x in root.iter('{http://www.w3.org/2000/svg}g'):
    print(x)

<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFA60>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFC90>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9440>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A94E0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A97B0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9C10>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9EE0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9FD0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AA390>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AA6B0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AA930>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AAAC0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AAC50>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AADE0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AAED0>
<Element '{http://www.w3.org/2000/svg}g'

In [20]:
for x in root.findall('svg:g', ns):
    print(x)

<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFA60>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFC90>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9440>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AB560>
